In [ ]:
import pandas as pd
import numpy as np
import random
import jieba

In [ ]:
data = pd.read_csv("17_19_Data.csv", encoding="gbk")
data = data[data.columns[:16]] # 一共有16个特征
data = data.drop(labels = 1772) # 发现第1772个数据在原数据集里列填串了，删去
data = data.drop(labels = 8297) # 发现第8297个数据无标签，删去
data.head()

In [ ]:
data = data.apply(lambda x: x.replace("\n", "")) # 去除换行符
data["rand"] = np.random.uniform(0, 1, len(data))
data = data.sort_values(by = "rand") # 对data随机排序
# data["rand"]

In [ ]:
# data2 = data[:400]
# data2
# data2.to_csv("rand_select_400.csv", encoding="utf-8") # data2为随机抽取的400个样本，用于数据细化

In [ ]:
# 院系、专业、民族、出生年月、校区、rand没有用，删掉
data = data.drop(['院系', '专业', '民族', '出生年月', '所在校区', 'rand'], axis=1)
data.head()

### 

In [ ]:
# 预处理完毕，开始对十个特征进行处理：
# 户口性质、家庭人口、年收入、是否贷款、贫困类型
# 国家资助，经济来源，受教育情况，突发事件情况，在校受资助情况

In [ ]:
# 享受国家政策资助情况

data["享受国家政策资助情况"] = data["享受国家政策资助情况"].fillna('无')

data["建档立卡贫困户"] = ''
index = data["享受国家政策资助情况"].str.contains("立卡")
data.loc[index, "建档立卡贫困户"] = 1
data.loc[-index, "建档立卡贫困户"] = 0

data["城乡低保户"] = ''
index = data["享受国家政策资助情况"].str.contains("低保")
data.loc[index, "城乡低保户"] = 1
data.loc[-index, "城乡低保户"] = 0

data["五保户"] = ''
index = data["享受国家政策资助情况"].str.contains("五保")
data.loc[index, "五保户"] = 1
data.loc[-index, "五保户"] = 0

data["农村特困供养"] = ''
index = data["享受国家政策资助情况"].str.contains("农村特困供养")
data.loc[index, "农村特困供养"] = 1
data.loc[-index, "农村特困供养"] = 0

data["孤残学生"] = ''
index = data["享受国家政策资助情况"].str.contains("孤残")
data.loc[index, "孤残学生"] = 1
data.loc[-index, "孤残学生"] = 0

data["军烈属或优抚子女"] = ''
index = data["享受国家政策资助情况"].str.contains("军烈属")
data.loc[index, "军烈属或优抚子女"] = 1
data.loc[-index, "军烈属或优抚子女"] = 0

# pd.set_option('max_row', 100)
# data.head()

In [ ]:
del data["享受国家政策资助情况"]
#data

In [ ]:
# 家庭主要经济来源，这一项每人只有一种选择（主要经济来源），因此不需要哑变量
data["家庭主要经济来源"] = data["家庭主要经济来源"].fillna('父母均下岗')

index = data["家庭主要经济来源"].str.contains("生意|经营|从商|经商|地摊|摆摊|杂货铺|店|卖|买|个体|餐|理发|手工|个体|水果摊|蒸馒头|股票")
data.loc[index, "家庭主要经济来源"] = "做生意"

index = (data["家庭主要经济来源"] == "城镇") | (data["家庭主要经济来源"] == "父母劳作") | (data["家庭主要经济来源"] == "家长工资") | (data["家庭主要经济来源"] == "父亲、母亲") | (data["家庭主要经济来源"] == "父母微薄收入") | (data["家庭主要经济来源"] == "工薪") | (data["家庭主要经济来源"] == "基本工资") | (data["家庭主要经济来源"] == "职工工资") | (data["家庭主要经济来源"] == "父母工资") | (data["家庭主要经济来源"] == "父母工作收入") | (data["家庭主要经济来源"] == "父母") | (data["家庭主要经济来源"] == "工作") | (data["家庭主要经济来源"] == "父母工作") | (data["家庭主要经济来源"] == "父母收入") | (data["家庭主要经济来源"] == "工资收入") | (data["家庭主要经济来源"] == "父母工资") | (data["家庭主要经济来源"] == "工资") | (data["家庭主要经济来源"].str.contains('打工|务工|农民工|工地|零工|临时工|工人|临工|短工|小工|散工|出租车|货车|教师|苦力|司机|体力劳动|保安|看守|送货|公交车|裁缝|保姆|上班|工活|教书|清洁工|营业员|城市|普通职工|诊所|超市工作|跑保险|打杂|干活|杂工|十字绣|代教|职员|瓷砖|看门|建房子|职工|房屋出租|房租|自由职业|副业|父母工资收入|父母劳动收入|劳务报酬|父母的工资|做工|劳动收入|卫生所从医|工作收入|不固定|不稳定|无稳定|非固定|非稳定|无固定|没有固定'))
data.loc[index, "家庭主要经济来源"] = "打工"

index = data["家庭主要经济来源"].str.contains('务农|农作|农民|农收|农业|农务|农村|农活|耕|种植|种地|种粮|庄稼|田|农产品|土地|葡萄|果树|果园|畜|玉米|梨园|牧|养殖|苹果|枣')
data.loc[index, "家庭主要经济来源"] = "务农"

index = (data["家庭主要经济来源"] == "未写") | (data["家庭主要经济来源"] == "暂无") | (data["家庭主要经济来源"] == "无") | (data["家庭主要经济来源"] == "兄长") | (data["家庭主要经济来源"] == "姐姐的工资") | (data["家庭主要经济来源"] == "哥哥工作") | (data["家庭主要经济来源"] == "姐姐 哥哥") | (data["家庭主要经济来源"] == "姐姐的工资收入") | (data["家庭主要经济来源"] == "哥哥工资") | (data["家庭主要经济来源"] == "本人及奶奶的低保金") | (data["家庭主要经济来源"] == "姐姐工资") | (data["家庭主要经济来源"] == "现靠父母过去的工资") | (data["家庭主要经济来源"] == "靠姑姑接济") | (data["家庭主要经济来源"] == "父亲无固定工作，现停业在家\n母亲一直无工作") | data["家庭主要经济来源"].str.contains('父母无业|双方失业|父母均无业|父母下岗|均下岗|双下岗|亲友|接济|救济|资助|勤工俭学|经济扶持|补助|补贴|寄养家庭|父母离岗工资|社保')
data.loc[index, "家庭主要经济来源"] = "父母均下岗"

index = data["家庭主要经济来源"].str.contains('低保|最低生活保障')
data.loc[index, "城乡低保户"] = 1 # 有些人在前面没写低保，应该加上
data.loc[index, "家庭主要经济来源"] = "父母均下岗"

index = (data["家庭主要经济来源"] == "父母一方下岗") | (data["家庭主要经济来源"] == "父亲每月工资") | (data["家庭主要经济来源"] == "母亲基本工资") | (data["家庭主要经济来源"] == "父亲的薪水") | (data["家庭主要经济来源"] == "父亲基本工资") | (data["家庭主要经济来源"] == "父亲和兄长收入") | (data["家庭主要经济来源"] == "父亲姐姐工资") | (data["家庭主要经济来源"] == "母亲单位工资") | (data["家庭主要经济来源"] == "父亲上岗") | (data["家庭主要经济来源"] == "父亲劳务派遣") | (data["家庭主要经济来源"] == "父亲固定工资收入") | (data["家庭主要经济来源"] == "父亲个人工资") | (data["家庭主要经济来源"] == "父亲微薄工资") | (data["家庭主要经济来源"] == "4050公益岗位收入") | (data["家庭主要经济来源"] == "父兄工资") | (data["家庭主要经济来源"] == "爸爸") | (data["家庭主要经济来源"] == "父亲的收入") | (data["家庭主要经济来源"] == "父亲的工作") | (data["家庭主要经济来源"] == "父亲") | (data["家庭主要经济来源"] == "母亲") | (data["家庭主要经济来源"].str.contains('一方|母亲固定收入|父亲固定收入|母亲下岗|父亲下岗|父亲失业|母亲失业|一人工资|一人的工资|爸爸工资|妈妈工资|爸爸的工资|妈妈的工资|父亲的工资|父亲工资|母亲的工资|母亲工资|父亲工作|母亲工作|爸爸工作|妈妈工作|父亲上班|母亲上班|父亲收入|母亲收入|父亲无业'))
data.loc[index, "家庭主要经济来源"] = "父母一方下岗"

index = data["家庭主要经济来源"].str.contains('退休|养老|退养|病休')
data.loc[index, "家庭主要经济来源"] = "退休金或养老金"

index = data["家庭主要经济来源"].str.contains('内退|病退|退职')
data.loc[index, "家庭主要经济来源"] = "退职金或病退金"

In [ ]:
# 家庭其他成员在受教育情况，统计结果为正在接受大学教育的人数、正在接受高中教育的人数、正在接受义务教育的人数
# 认为空值、0均为“无”，先筛选出“无”的
data["家庭其他成员在受教育情况"] = data["家庭其他成员在受教育情况"].fillna('无')

index = (data["家庭其他成员在受教育情况"] == "0") | (data["家庭其他成员在受教育情况"] == "0人") | (data["家庭其他成员在受教育情况"] == "0人在读高中或大学") | data["家庭其他成员在受教育情况"].str.contains("暂无|独生|无在读|无在受|无其他|无成员|无高中|无正在")
data.loc[index, "家庭其他成员在受教育情况"] = "无"

data[data["家庭其他成员在受教育情况"] != "无"] # 还有4331个

In [ ]:
def education(strs):
    # 创建关键字
    number1 = {"一个", "1个", "一人", "1人", "一位", "1位"}
    number2 = {"两个", "2个", "两人", "2人", "二人", "两位", "2位", "二位"}
    number3 = {"三个", "3个", "三人", "3人", "三位", "3位"}
    number4 = {"四个", "4个", "四人", "4人", "四位", "4位"}

    member = {"哥哥", "姐姐", "弟弟", "妹妹", "侄女", "侄子"}
    sp_member = {"哥", "兄", "姐", "弟", "妹", "大哥", "二哥", "大弟", "二弟", "三弟", "四弟", "五弟", "小弟", "大姐", "长姐", "二姐", "三姐", "大妹", "小妹", "二妹", "三妹", "四妹"}
    invalid_member = {"爸爸", "父亲", "妈妈", "母亲", "爷爷", "奶奶", "外祖父", "外祖母", "姥爷", "姥姥", "伯伯", "婆婆", "外公", "外婆"}
    invalid_sp_member = {"爸", "妈", "爷", "奶", "祖父", "祖母", "父", "母"}

    grad = {"幼儿园毕业", "刚毕业", "小学毕业", "小学未毕业", "初中毕业", "初中未毕业", "高中毕业", "大专毕业", "专科毕业", "大学毕业", "三本毕业", "本科毕业", "应届毕业", "大学已毕业", "大学刚毕业", "本科毕业", "研究生毕业", "硕士毕业", "博士毕业"}
    sp_grad = {"毕业", "未受教育", "未接受教育", "文盲", "无学历", "没上过学", "肄业", "未上学"}
    college = {"研究生", "读研", "博士", "硕士", "大学", "本科", "专升本", "大学生", "河工大"}
    sp_college = {"考研", "大专", "学院", "高职", "专科", "职业技术学校"}
    gr_college = {"大一", "大二", "大三", "大四", "研一", "研二", "研三", "博二", "读博"}
    high_school = {"高中", "高专", "职高", "职专", "职中", "中专", "中学", "职业高中"}
    gr_high_school = {"高一", "高二", "高三", "高考"}

    compulsory = {"初中", "小学"}
    gr_compulsory = {"初一", "初二", "初三", "初中三年级", "一年级", "二年级", "三年级", "四年级", "五年级", "六年级", "七年级", "八年级", "九年级", "义务教育阶段"}
    others = {"幼儿园", "学前班", "学前教育"}

    # 手动检查了前200个数据，发现一些分词结果有误
    jieba.suggest_freq(('父', '母'), True)
    jieba.suggest_freq(('兄弟'), True)
    jieba.suggest_freq(('兄', '妹'), True)
    jieba.suggest_freq(('姐', '弟'), True)
    jieba.suggest_freq(('姐', '妹'), True)
    jieba.suggest_freq(('读', '高二'), True)
    jieba.suggest_freq(('高二'), True)
    jieba.suggest_freq(('小学毕业'), True)
    jieba.suggest_freq(('初中毕业'), True)
    jieba.suggest_freq(('高中毕业'), True)
    jieba.suggest_freq(('职高毕业'), True)
    jieba.suggest_freq(('大学毕业'), True)
    jieba.suggest_freq(('大专毕业'), True)
    jieba.suggest_freq(('1人'), True)
    jieba.suggest_freq(('1个'), True)
    jieba.suggest_freq(('1位'), True)
    jieba.suggest_freq(('2人'), True)
    jieba.suggest_freq(('2个'), True)
    jieba.suggest_freq(('2位'), True)
    jieba.suggest_freq(('3人'), True)
    jieba.suggest_freq(('3个'), True)
    jieba.suggest_freq(('3位'), True)
    jieba.suggest_freq(('4人'), True)
    jieba.suggest_freq(('中医药', '大学'), True)
    jieba.suggest_freq(('初中三年级'), True)
    jieba.suggest_freq(('西安交通大学'), True)
    jieba.suggest_freq(('海南师范大学'), True)
    jieba.suggest_freq(('河工大'), True)
    jieba.suggest_freq(('中国科学院'), True)
    jieba.suggest_freq(('北京航空航天大学'), True)
    jieba.suggest_freq(('北京大学'), True)
    jieba.suggest_freq(('农', '学院'), True)
    jieba.suggest_freq(('义务教育阶段'), True)
    
    # 对每个字符串进行分词，并且找寻其中关键字，记录关键字的词性和位置
    arr = np.zeros(shape = (len(strs), 3))
    df = pd.DataFrame(arr, columns = ["大学", "高中", "义务教育"])
    row = -1 # 记录str在strs中的位置

    for str in strs:
        row += 1
        seg_list = jieba.cut(str, cut_all = False) # 对每个字符串进行分词
        output = list(seg_list)
        cut_type = [] # 记录某个关键字的词性
        cut_loc = [] # 记录某个关键字在str中的位置
        loc = 0 # loc为某个词在原字符串中的位置(1~len)

        for cut in output: # cut为分词结果中的每个词
            loc += 1
            if (cut in number1):
                cut_type.append("number1")
                cut_loc.append(loc-1) # cut在原字符串中的索引值(0~len-1)

            elif (cut in number2):
                cut_type.append("number2")
                cut_loc.append(loc-1)

            elif (cut in number3):
                cut_type.append("number3")
                cut_loc.append(loc-1)

            elif (cut in number4):
                cut_type.append("number4")
                cut_loc.append(loc-1)

            elif (cut in member):
                cut_type.append("member")
                cut_loc.append(loc-1)

            elif (cut in sp_member):
                cut_type.append("sp_member")
                cut_loc.append(loc-1)

            elif (cut in invalid_member):
                cut_type.append("invalid_member")
                cut_loc.append(loc-1)

            elif (cut in invalid_sp_member):
                cut_type.append("invalid_sp_member")
                cut_loc.append(loc-1)

            elif (cut in grad):
                cut_type.append("grad")
                cut_loc.append(loc-1)

            elif (cut in sp_grad):
                cut_type.append("sp_grad")
                cut_loc.append(loc-1)

            elif (cut in college):
                cut_type.append("college")
                cut_loc.append(loc-1)

            elif (cut in sp_college):
                cut_type.append("sp_college")
                cut_loc.append(loc-1)

            elif (cut in gr_college):
                cut_type.append("gr_college")
                cut_loc.append(loc-1)

            elif (cut in high_school):
                cut_type.append("high_school")
                cut_loc.append(loc-1)

            elif (cut in gr_high_school):
                cut_type.append("gr_high_school")
                cut_loc.append(loc-1)

            elif (cut in compulsory):
                cut_type.append("compulsory")
                cut_loc.append(loc-1)

            elif (cut in gr_compulsory):
                cut_type.append("gr_compulsory")
                cut_loc.append(loc-1)

            elif (cut in others):
                cut_type.append("others")
                cut_loc.append(loc-1)

#         print(str)
#         print(output)
#         print(cut_type)
#         print(cut_loc)

#         在cut_type中找寻如下pattern，并总结出大学阶段、高中阶段、义务教育阶段各有多少人：
#         个数和家庭成员都有可能出现，但个数为家庭成员前一个词，因此先检测个数再紧跟着检测家庭成员
#         年级和学校都有可能出现，但年级一定出现在学校之后，因此先检测学校再检测年级
#         个数 -> 学校/年级/学校&年级 -> 非学校或年级：这几个人都属于该学校
#         （个数 ->）家庭成员 -> 学校/年级/学校&年级 -> 非学校或年级：这几个人都属于该学校
#         （个数 ->）家庭成员 -> 学校/年级/学校&年级 -> 学校/年级/学校&年级 -> 非学校或年级：首先保证两个学校阶段相同，则这种家庭成员分别属于这个阶段；否则人工处理

        serial = 0
        cut_type.append(" ")
        cut_type.append(" ") # 确保检测到最后一位也能检测其后两位的元素的词性
        u_num = 0 # 大学生人数
        h_num = 0 # 高中生人数
        c_num = 0 # 义务教育人数
        number = 0 # 待定的人数
        while(serial < len(cut_type)-2):
            current_group = []
            # 每一个pattern起始的词只能为个数或家庭成员
            if(cut_type[serial] in ["number1", "number2", "number3", "number4", "member", "sp_member", "invalid_member", "invalid_sp_member"]):
                current_group.append(cut_type[serial])
                for forward in range(serial+1, len(cut_type)-1): # 开始检测该pattern内后面的词
                    current_group.append(cut_type[forward])
                    if((cut_type[forward] in [" ", "grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"]) and (cut_type[forward+1] in [" ","number1", "number2", "number3", "number4", "member", "sp_member", "invalid_member", "invalid_sp_member"])):
                        serial = forward + 1 # 找到学校或年级 -> 非学校或年级，这一组完成，定位至下一组首个词
                        break

                # 此时一组已经检测完成，对其进行匹配
                group_serial = 0
                current_group.append(" ")
                current_group.append(" ") # 确保检测到最后一位也能检测其后两位的元素是否为学校

                while(group_serial < len(current_group)-2):
                    # 个数 -> 学校 -> 非学校或年级
                    if ((current_group[group_serial] in ["number1", "number2", "number3", "number4"]) and (current_group[group_serial+1] in ["college", "sp_college", "gr_college"])):
                        if(current_group[group_serial] == "number1"):
                            u_num += 1
                        elif(current_group[group_serial] == "number2"):
                            u_num += 2
                        elif(current_group[group_serial] == "number3"):
                            u_num += 3
                        elif(current_group[group_serial] == "number4"):
                            u_num += 4

                    elif ((current_group[group_serial] in ["number1", "number2", "number3", "number4"]) and (current_group[group_serial+1] in ["high_school", "gr_high_school"])):
                        if(current_group[group_serial] == "number1"):
                            h_num += 1
                        elif(current_group[group_serial] == "number2"):
                            h_num += 2
                        elif(current_group[group_serial] == "number3"):
                            h_num += 3
                        elif(current_group[group_serial] == "number4"):
                            h_num += 4

                    elif ((current_group[group_serial] in ["number1", "number2", "number3", "number4"]) and (current_group[group_serial+1] in ["compulsory", "gr_compulsory"])):
                        if(current_group[group_serial] == "number1"):
                            c_num += 1
                        elif(current_group[group_serial] == "number2"):
                            c_num += 2
                        elif(current_group[group_serial] == "number3"):
                            c_num += 3
                        elif(current_group[group_serial] == "number4"):
                            c_num += 4

                    # (个数 ->) 家庭成员 -> 学校 -> 非学校
                    elif ((current_group[group_serial] in ["member", "sp_member"]) and (current_group[group_serial+1] in ["grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"]) and (current_group[group_serial+2] not in ["grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"])):
                        if(current_group[group_serial-1] == "number2"):
                            number += 2
                        elif(current_group[group_serial-1] == "number3"):
                            number += 3
                        elif(current_group[group_serial-1] == "number4"):
                            number += 4
                        else:
                            number += 1

                        if(current_group[group_serial+1] in ["college", "sp_college", "gr_college"]):
                            u_num += number
                        elif(current_group[group_serial+1] in ["high_school", "gr_high_school"]):
                            h_num += number
                        elif(current_group[group_serial+1] in ["compulsory", "gr_compulsory"]):
                            c_num += number

                        number = 0

                    # 家庭成员 -> 多个学校 -> 非学校
                    elif ((current_group[group_serial] in ["member", "sp_member"]) and (current_group[group_serial+1] in ["grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"]) and (current_group[group_serial+2] in ["grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"])):
                        group_serial += 1
                        while (group_serial < len(current_group)-2):
                            if (current_group[group_serial] not in ["grad", "sp_grad", "college", "sp_college", "gr_college", "high_school", "gr_high_school", "compulsory", "gr_compulsory", "others"]):
                                break

                            elif(current_group[group_serial] in ["college", "sp_college", "gr_college"]):
                                u_num += 1
                            elif(current_group[group_serial] in ["high_school", "gr_high_school"]):
                                h_num += 1
                            elif(current_group[group_serial] in ["compulsory", "gr_compulsory"]):
                                c_num += 1
                            group_serial += 1

                    # 多个家庭成员 -> 学校 -> 非学校
                    elif ((current_group[group_serial] in ["member", "sp_member"]) and (current_group[group_serial+1] in ["member", "sp_member", "invalid_member", "invalid_sp_member"])):
                        group_serial += 1
                        while (group_serial < len(current_group)-2):
                            if(current_group[group_serial] not in ["member", "sp_member", "invalid_member", "invalid_sp_member"]):
                                break
                            elif(current_group[group_serial] in ["member", "sp_member"]):
                                number += 1
                            group_serial += 1

                        if(current_group[group_serial] in ["college", "sp_college", "gr_college"]):
                            u_num += number
                        elif(current_group[group_serial] in ["high_school", "gr_high_school"]):
                            h_num += number
                        elif(current_group[group_serial] in ["compulsory", "gr_compulsory"]):
                            c_num += number

                        number = 0

                    group_serial += 1

            else: # 该词不为个数或家庭成员，则访问下一个
                serial += 1

        df.iloc[row] = [u_num, h_num, c_num]
    
    return(df)

In [ ]:
strs = data["家庭其他成员在受教育情况"]
df = education(strs)
df

In [ ]:
# 突发事件情况
data["突发事件情况"] = data["突发事件情况"].fillna('无')

index = (data["突发事件情况"] == "否") | (data["突发事件情况"] == "没有") | (data["突发事件情况"] == "正常") | (data["突发事件情况"] == "无") | (data["突发事件情况"] == "暂无") | (data["突发事件情况"] == "自然状况")
data.loc[index, "突发事件情况"] = "无"

#发现有人在这里填自己是孤残学生，加到孤残学生中
index = data["突发事件情况"].str.contains("父母双亡|父母去世|孤残|孤儿|重大疾病、突发意外致残|本人视力残疾|本人严重烫伤")
data.loc[index, "孤残学生"] = 1

data[data["突发事件情况"] != "无"] # 还有4142个

In [ ]:
def accidents(strs):
    # 创建关键字
    dad = {"爸爸", "父亲", "爸", "父"}
    mom = {"妈妈", "母亲", "妈", "母"}
    grand_parents = {"老人", "长辈", "祖父母", "爷爷", "奶奶", "外祖父", "外祖母", "姥爷", "姥姥", "外公", "外婆"}
    sp_grand_parents = {"爷", "奶", "祖父", "祖母"}
    siblings = {"哥哥", "姐姐", "弟弟", "妹妹"}
    sp_siblings = {"哥", "兄", "姐", "弟", "妹", "大哥", "二哥", "大弟", "二弟", "三弟", "四弟", "五弟", "小弟", "大姐", "长姐", "二姐", "三姐", "大妹", "小妹", "二妹", "三妹", "四妹"}
    invalid_member = {"我", "本人", "自己", "侄女", "侄子", "伯伯", "伯母", "大伯", "二伯", "三伯", "伯父", "婆婆", "舅舅", "小姑", "姑姑", "二姑", "大舅", "叔叔", "叔父", "二叔", "老叔", "舅妈"}

    divorce = {"单亲", "离婚", "离异"}
    unemployed = {"无业", "一方无业", "均无业", "失业", "无法工作", "下岗", "公司破产", "无工作", "待业", "倒闭", "离职", "未有收入", "无收入", "停产", "失去稳定工作", "没能工作"}
    dead = {"去世", "离世", "病逝", "死亡", "治丧", "身亡", "病故"}
    illness = {"病", "病了", "就医", "发病", "有病", "多病", "车祸", "住院", "养病", "疾病", "病情", "受伤", "顽疾", "服药", "腰伤", "慢性疾病", "普通疾病", "一般疾病", "生病", "患病", "带病", "患疾", "皮肤病", "高血压", "高血糖", "高血脂", "风湿", "类风湿", "风湿病", "心脏病", "糖尿病", "高血脂", "三高", "囊肿", "肝囊肿", "结石", "肾结石", "胆结石", "结石病", "尿结石", "肾囊肿", "肾积水", "脑溢血", "脑血栓", "心脑血管疾病", "心脑疾病", "青光眼", "慢阻肺", "中风", "白内障", "肺结核", "冠心病", "甲亢", "癫痫", "股骨头坏死", "风湿", "腿脚不便", "精神病", "精神疾病", "精神分裂症", "精神性疾病", "气胸", "胃穿孔", "骨折", "骨裂", "红斑狼疮", "腰椎间盘突出", "腰间盘突出", "关节炎", "骨质增生", "胃溃疡", "手术", "腿疾", "胃病", "感染", "胰腺炎", "溃烂", "摔伤", "腿伤", "睡眠障碍", "工伤", "视网膜", "白癜风", "关节病", "颈椎病", "胆囊炎", "坠楼", "瘸", "摔伤", "腿伤", "贫血", "脱髓鞘", "事故", "意外事故", "服药", "体弱", "卷入机器", "气管炎", "支气管炎", "卧病", "交通事故", "吃药", "胃出血", "脑出血", "颅内出血", "子宫肌瘤", "腰椎", "颈椎", "腰椎病", "后遗症", "割伤", "脑垂体瘤", "脊椎炎", "扎伤", "烫伤", "肺气肿", "卧床", "断裂", "眼疾", "伤手", "摔了", "吃药", "旧病复发", "切断", "摔到", "意外事故"}
    serious_illness = {"大病", "病重", "重病", "重疾", "重大疾病", "肌无力", "肿瘤", "瘤", "白血病", "癌", "患癌", "癌症", "肝癌", "食道癌", "卵巢癌", "甲状腺癌", "肺癌", "宫颈癌", "脑癌", "直肠癌", "乳腺癌", "胃癌", "肺腺癌", "贲门癌", "喷门癌", "食道癌", "肠癌", "乳癌", "结肠癌", "前列腺癌", "致癌", "肾癌", "淋巴癌", "食道癌", "心梗", "心肌梗塞", "脑中风", "移植", "搭桥", "支架", "肾炎", "肾病综合征", "肾综合", "严重肾病", "肾衰竭", "尿毒症", "截肢", "肝病", "肝硬化", "肝炎", "干重活", "做重活", "不能工作", "失去劳动力", "丧失劳动力", "丧失行动力", "无法劳作", "丧失劳动能力", "失去部分劳动力", "失去全部劳动力", "无法承受过重劳动", "失去行动能力", "干不了", "干重活", "不能劳作", "不得剧烈运动", "脑梗", "脑梗塞", "脑梗死", "脑膜炎", "脑膜瘤", "昏迷", "聋", "失聪", "耳聋", "聋哑人", "聋哑", "失明", "瘫痪", "偏瘫", "脑瘫", "截瘫", "致瘫", "帕金森", "瓣膜病", "痴呆", "老年痴呆", "老年痴呆症", "烧伤", "火烧", "语言", "贫血", "主动脉", "残疾", "残废", "伤残", "摔断", "砸断", "神志不清", "骨髓瘤", "致残", "脑萎缩", "脑血管", "脑结核", "半身不遂", "致盲", "病危", "再生性贫血障碍", "生活无法自理", "做手术"}

    jieba.suggest_freq(('严重肾病'), True)
    jieba.suggest_freq(('肾病综合征'), True)
    jieba.suggest_freq(('失去劳动力'), True)
    jieba.suggest_freq(('丧失劳动力'), True)
    jieba.suggest_freq(('丧失行动力'), True)
    jieba.suggest_freq(('无法劳作'), True)
    jieba.suggest_freq(('不能劳作'), True)
    jieba.suggest_freq(('失去部分劳动力'), True)
    jieba.suggest_freq(('失去部分劳动力'), True)
    jieba.suggest_freq(('失去行动能力'), True)
    jieba.suggest_freq(('丧失劳动能力'), True)
    jieba.suggest_freq(('无法承受过重劳动'), True)
    jieba.suggest_freq(('心脑血管疾病'), True)
    jieba.suggest_freq(('心脑疾病'), True)
    jieba.suggest_freq(('肺腺癌'), True)
    jieba.suggest_freq(('脑梗'), True)
    jieba.suggest_freq(('不得剧烈运动'), True)
    jieba.suggest_freq(('重大疾病'), True)
    jieba.suggest_freq(('癫痫', '病'), True)
    jieba.suggest_freq(('普通疾病'), True)
    jieba.suggest_freq(('一般疾病'), True)
    jieba.suggest_freq(('一方无业'), True)
    jieba.suggest_freq(('均无业'), True)
    jieba.suggest_freq(('股骨头坏死'), True)
    jieba.suggest_freq(('干重活'), True)
    jieba.suggest_freq(('做重活'), True)
    jieba.suggest_freq(('腿脚不便'), True)
    jieba.suggest_freq(('无法工作'), True)
    jieba.suggest_freq(('工作'), True)
    jieba.suggest_freq(('精神性疾病'), True)
    jieba.suggest_freq(('精神官能症'), True)
    jieba.suggest_freq(('单亲', '家庭'), True)
    jieba.suggest_freq(('恶性', '肿瘤'), True)
    jieba.suggest_freq(('断', '腿'), True)
    jieba.suggest_freq(('断', '手'), True)
    jieba.suggest_freq(('公司破产'), True)
    jieba.suggest_freq(('腰椎间盘突出'), True)
    jieba.suggest_freq(('腰间盘突出'), True)
    jieba.suggest_freq(('腿疾'), True)
    jieba.suggest_freq(('无工作'), True)
    jieba.suggest_freq(('慢性疾病'), True)
    jieba.suggest_freq(('睡眠障碍'), True)
    jieba.suggest_freq(('家', '父'), True)
    jieba.suggest_freq(('家', '母'), True)
    jieba.suggest_freq(('卷入机器'), True)
    jieba.suggest_freq(('颅内出血'), True)
    jieba.suggest_freq(('子宫肌瘤'), True)
    jieba.suggest_freq(('未有收入'), True)
    jieba.suggest_freq(('脑垂体瘤'), True)
    jieba.suggest_freq(('卧床', '不起'), True)
    jieba.suggest_freq(('黑色素', '瘤'), True)
    jieba.suggest_freq(('病了'), True)
    jieba.suggest_freq(('再生性贫血障碍'), True)
    jieba.suggest_freq(('摔了'), True)
    jieba.suggest_freq(('生活无法自理'), True)
    jieba.suggest_freq(('失去稳定工作'), True)
    jieba.suggest_freq(('没能工作'), True)
    jieba.suggest_freq(('旧病复发'), True)
    jieba.suggest_freq(('摔到'), True)

    # 对每个字符串进行分词，并且找寻其中关键字，记录关键字的词性和位置
    arr = np.zeros(shape = (len(strs), 11))
    df = pd.DataFrame(arr, columns=["祖父母患病", "父母离异", "父亲（母亲）患普通疾病", "父母患普通疾病", "父亲（母亲）无业", "父母均无业", "兄弟姐妹患重疾", "父亲（母亲）患重疾", "父母患重疾", "父亲（母亲）去世", "突发重大自然灾害"])
    index = strs.str.contains("灾|病虫害|霜冻|地震|台风|洪水|大水|大旱|干旱|冰雹|暴风雨|暴雨|下雪|雷劈|禽流感|高温|减产|倒伏|淹|涝|庄稼大量死亡|自然状况|自然天气状况|减产|泥石流|猪瘟|庄稼无收")

    adjusted_index = 0 # 由于index中少了一些索引，无法直接匹配到df中，需要调整index
    for i in index:
        if (i):
            df.loc[adjusted_index, "突发重大自然灾害"] = 1
        adjusted_index += 1

    row = -1 # 记录str在strs中的位置

    for str in strs:
        row += 1 
        seg_list = jieba.cut(str, cut_all = False) # 对每个字符串进行分词
        output = list(seg_list)
        cut_type = [] # 记录某个关键字的词性
        cut_loc = [] # 记录某个关键字在str中的位置
        loc = 0 # loc为某个词在原字符串中的位置(1~len)

        for cut in output: # cut为分词结果中的每个词
            loc += 1
            if (cut in dad):
                cut_type.append("dad")
                cut_loc.append(loc-1) # cut在原字符串中的索引值(0~len-1)

            elif (cut in mom):
                cut_type.append("mom")
                cut_loc.append(loc-1)

            elif (cut in grand_parents):
                cut_type.append("grand_parents")
                cut_loc.append(loc-1)

            elif (cut in sp_grand_parents):
                cut_type.append("sp_grand_parents")
                cut_loc.append(loc-1)

            elif (cut in siblings):
                cut_type.append("siblings")
                cut_loc.append(loc-1)

            elif (cut in sp_siblings):
                cut_type.append("sp_siblings")
                cut_loc.append(loc-1)

            elif (cut in invalid_member):
                cut_type.append("invalid_member")
                cut_loc.append(loc-1)

            elif (cut in divorce):
                cut_type.append("divorce")
                cut_loc.append(loc-1)

            elif (cut in unemployed):
                cut_type.append("unemployed")
                cut_loc.append(loc-1)

            elif (cut in dead):
                cut_type.append("dead")
                cut_loc.append(loc-1)

            elif (cut in illness):
                cut_type.append("illness")
                cut_loc.append(loc-1)

            elif (cut in serious_illness):
                cut_type.append("serious_illness")
                cut_loc.append(loc-1)

#             print(str)
#             print(output)
#             print(cut_type)
#             print(cut_loc)

#         在cut_type中找寻如下pattern，并总结出每个人发生什么事情：
#         每一个人可能同时做多件事情，而这多件事情肯定是按顺序排列的，这些事情之间一定不出现另一个人名
#         无论在哪里出现divorce，一定为父母离异，但是如果出现在“父母”之后，则需要把这个“父母”与divorce绑定
#         这里jieba无法将“父母”、“父/母”、“父(母)”、“父亲（母亲）”分开，所以需要加一个判断条件，会用在后面无业、患病、去世中
#         一个人之后可能跟着多个illness，应全部与其绑定。若人是祖父母，则统计其是否患病；父母则看是否有重病，且应将父母辨别开；兄弟姐妹只统计重疾。
#         有可能出现人 -> illness ->dead。所有人都有可能dead，dead需要与之前最近的一个人或连续的多个人绑定，但只统计父或母去世。

        serial = 0
        cut_type.append(" ")
        cut_type.append(" ") # 确保检测到最后一位也能检测其后两位的元素的词性
        cut_loc.append(" ")
        cut_loc.append(" ")

        while(serial < len(cut_type)-2):
            flag_dad = False
            flag_mom = False
            flag_parents = False # 判断是父或母还是父与母
            flag_grand_parents = False
            flag_sp_grand_parents = False
            flag_siblings = False
            flag_sp_siblings = False
            flag_divorce = False
            flag_unemployed = False
            flag_dead = False
            flag_illness = False
            flag_serious_illness = False #用来检测每组中是否出现相应的关键词，若有则按照规则赋予相应的类1

            current_group = []
            current_loc = []

            # 每一个pattern起始的词只能为家庭成员
            if(cut_type[serial] in ["dad", "mom", "grand_parents", "sp_grand_parents", "siblings", "sp_siblings", "invalid_member"]):
                current_group.append(cut_type[serial])
                current_loc.append(cut_loc[serial]) # 用来判断父母是否连着
                
                for forward in range(serial+1, len(cut_type)-1): # 开始检测该pattern内后面的词
                    current_group.append(cut_type[forward])
                    current_loc.append(cut_loc[forward])
                    if((cut_type[forward] in [" ","divorce", "unemployed", "dead", "illness", "serious_illness"]) and (cut_type[forward+1] in [" ", "dad", "mom", "grand_parents", "sp_grand_parents", "siblings", "sp_siblings", "invalid_member"])):
                        serial = forward + 1 # 找到行为 -> 人，这一组完成，定位至下一组首个词
                        break
                        
                # 此时一组已经检测完成，对其进行匹配
                group_serial = 0
                current_group.append(" ")
                current_group.append(" ") # 确保检测到最后一位也能检测其后两位的元素
                current_loc.append(" ")
                current_loc.append(" ")
                current_loc.append(" ")
                current_loc.append(" ")
                
                while(group_serial < len(current_group)-2):
                    # 一个或多个家庭成员 -> 一件或多件事情 -> 非事情
                    if (current_group[group_serial] in ["dad", "mom", "grand_parents", "sp_grand_parents", "siblings", "sp_siblings", "invalid_member", "divorce", "unemployed", "dead", "illness", "serious_illness"]):
                        if (current_group[group_serial+1] not in [" ", "dad", "mom", "grand_parents", "sp_grand_parents", "siblings", "sp_siblings", "invalid_member", "divorce", "unemployed", "dead", "illness", "serious_illness"]):
                            break

                        if(current_group[group_serial] == "dad"):
                            flag_dad = True
                            if((current_group[group_serial+1] == "mom") and (current_loc[group_serial+1] == current_loc[group_serial]+1)):
                                flag_parents = True

                        elif(current_group[group_serial] == "mom"):
                            flag_mom = True
                            if((current_group[group_serial+1] == "dad") and (current_loc[group_serial+1] == current_loc[group_serial]+1)):
                                flag_parents = True

                        elif(current_group[group_serial] == "grand_parents"):
                            flag_grand_parents = True

                        elif(current_group[group_serial] == "sp_grand_parents"):
                            flag_sp_grand_parents = True

                        elif(current_group[group_serial] == "siblings"):
                            flag_siblings = True

                        elif(current_group[group_serial] == "sp_siblings"):
                            flag_sp_siblings = True

                        elif(current_group[group_serial] == "divorce"):
                            flag_divorce = True

                        elif(current_group[group_serial] == "unemployed"):
                            flag_unemployed = True

                        elif(current_group[group_serial] == "dead"):
                            flag_dead = True

                        elif(current_group[group_serial] == "illness"):
                            flag_illness = True

                        elif(current_group[group_serial] == "serious_illness"):
                            flag_serious_illness = True

                        # 一个pattern中的词已经全部找到，对该pattern进行考察
                        if((flag_grand_parents or flag_sp_grand_parents) and (flag_illness or flag_serious_illness)):
                            df.loc[row, "祖父母患病"] = 1

                        if(flag_divorce):
                            df.loc[row, "父母离异"] = 1

                        if(flag_illness and (flag_dad or flag_mom) and not flag_parents):
                            df.loc[row, "父亲（母亲）患普通疾病"] = 1

                        if(flag_illness and flag_dad and flag_mom and flag_parents):
                            df.loc[row, "父母患普通疾病"] = 1

                        if(flag_unemployed and (flag_dad or flag_mom) and not flag_parents):
                            df.loc[row, "父亲（母亲）无业"] = 1

                        if(flag_unemployed and flag_dad and flag_mom and flag_parents):
                            df.loc[row, "父母均无业"] = 1

                        if(flag_serious_illness and (flag_siblings or flag_sp_siblings)):
                            df.loc[row, "兄弟姐妹患重疾"] = 1

                        if(flag_serious_illness and (flag_dad or flag_mom) and not flag_parents):
                            df.loc[row, "父亲（母亲）患重疾"] = 1

                        if(flag_serious_illness and flag_dad and flag_mom and flag_parents):
                            df.loc[row, "父母患重疾"] = 1

                        if(flag_dead and (flag_dad or flag_mom) and not flag_parents):
                            df.loc[row, "父亲（母亲）去世"] = 1

                    group_serial += 1

            else: # 该词不为个数或家庭成员，则访问下一个
                serial += 1

    return(df)


In [ ]:
strs = data["突发事件情况"]
strs = pd.Series(data = strs)
df = accidents(strs)
df

In [ ]:
# 在校受奖励资助情况
data["在校受奖励资助情况"] = data["在校受奖励资助情况"].fillna('无')

index = data["在校受奖励资助情况"].str.contains("圆梦启航")
data.loc[index, "圆梦启航"] = 1
data.loc[-index, "圆梦启航"] = 0

index = data["在校受奖励资助情况"].str.contains("慧明")
data.loc[index, "慧明"] = 1
data.loc[-index, "慧明"] = 0

index = data["在校受奖励资助情况"].str.contains("欧莱雅")
data.loc[index, "欧莱雅"] = 1
data.loc[-index, "欧莱雅"] = 0

index = data["在校受奖励资助情况"].str.contains("喜来健")
data.loc[index, "喜来健"] = 1
data.loc[-index, "喜来健"] = 0

index = data["在校受奖励资助情况"].str.contains("中海油")
data.loc[index, "中海油"] = 1
data.loc[-index, "中海油"] = 0

index = data["在校受奖励资助情况"].str.contains("承锋")
data.loc[index, "承锋"] = 1
data.loc[-index, "承锋"] = 0

index = data["在校受奖励资助情况"].str.contains("宋声扬")
data.loc[index, "宋声扬"] = 1
data.loc[-index, "宋声扬"] = 0

index = data["在校受奖励资助情况"].str.contains("国泰")
data.loc[index, "国泰"] = 1
data.loc[-index, "国泰"] = 0

index = data["在校受奖励资助情况"].str.contains("思源")
data.loc[index, "思源"] = 1
data.loc[-index, "思源"] = 0

index = data["在校受奖励资助情况"].str.contains("清茗雅轩")
data.loc[index, "清茗雅轩"] = 1
data.loc[-index, "清茗雅轩"] = 0

index = data["在校受奖励资助情况"].str.contains("盛帆")
data.loc[index, "盛帆"] = 1
data.loc[-index, "盛帆"] = 0

index = data["在校受奖励资助情况"].str.contains("长城")
data.loc[index, "长城"] = 1
data.loc[-index, "长城"] = 0

index = data["在校受奖励资助情况"].str.contains("电装")
data.loc[index, "电装"] = 1
data.loc[-index, "电装"] = 0

index = data["在校受奖励资助情况"].str.contains("交通")
data.loc[index, "交通"] = 1
data.loc[-index, "交通"] = 0

index = data["在校受奖励资助情况"].str.contains("福慧")
data.loc[index, "福慧"] = 1
data.loc[-index, "福慧"] = 0

index = data["在校受奖励资助情况"].str.contains("柏年")
data.loc[index, "柏年"] = 1
data.loc[-index, "柏年"] = 0

index = data["在校受奖励资助情况"].str.contains("圣恩纳")
data.loc[index, "圣恩纳"] = 1
data.loc[-index, "圣恩纳"] = 0

index = data["在校受奖励资助情况"].str.contains("香港好友")
data.loc[index, "香港好友"] = 1
data.loc[-index, "香港好友"] = 0

index = data["在校受奖励资助情况"].str.contains("冯顾丽华")
data.loc[index, "冯顾丽华"] = 1
data.loc[-index, "冯顾丽华"] = 0

index = ((data["在校受奖励资助情况"].str.contains("国家")) | (data["在校受奖励资助情况"].str.contains("国助"))) & (data["在校受奖励资助情况"].str.contains("一"))
data.loc[index, "国家一等助学金"] = 1
data.loc[-index, "国家一等助学金"] = 0

index = ((data["在校受奖励资助情况"].str.contains("国家")) | (data["在校受奖励资助情况"].str.contains("国助"))) & (~ (data["在校受奖励资助情况"].str.contains("一")))
data.loc[index, "国家二等助学金"] = 1
data.loc[-index, "国家二等助学金"] = 0